In [ ]:
##########################################################
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import vgg16
from tensorflow.keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
import scipy  #Used to upsample our image

import os
import cv2
from PIL import Image
import numpy as np
from keras.applications.vgg16 import preprocess_input

In [ ]:

#Define the model. 
#Here, we use pre-trained VGG16 layers and add GlobalAveragePooling and dense prediction layers.
#You can define any model. 
#Also, here we set the first few convolutional blocks as non-trainable and only train the last block.
#This is just to speed up the training. You can train all layers if you want. 
def get_model(input_shape = (224,224,3)):
    
    vgg = vgg16.VGG16(weights='imagenet', include_top=False, input_shape = input_shape)

    #for layer in vgg.layers[:-8]:  #Set block4 and block5 to be trainable. 
    for layer in vgg.layers[:-5]:    #Set block5 trainable, all others as non-trainable
        print(layer.name)
        layer.trainable = False #All others as non-trainable.

    x = vgg.output
    x = GlobalAveragePooling2D()(x) #Use GlobalAveragePooling and NOT flatten. 
    x = Dense(2, activation="softmax")(x)  #We are defining this as multiclass problem. 
    
    model = Model(vgg.input, x)
    model.compile(loss = "categorical_crossentropy", 
                  optimizer = SGD(learning_rate=0.0001, momentum=0.9),
                  metrics=["accuracy"])
    
    return model

model = get_model(input_shape = (224,224,3))
print(model.summary())


In [ ]:
directory = '/content/drive/MyDrive/Fire Dataset PCD/'
batch_size = 16
target_size = (224,224)

from keras.applications.vgg16 import preprocess_input
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator=datagen.flow_from_directory(
    directory=directory + 'Train',
    batch_size=batch_size,
    target_size= target_size)


validation_generator=datagen.flow_from_directory(
    directory=directory + 'Test',
    batch_size=batch_size,
    target_size= target_size)


In [ ]:
history = model.fit(train_generator,
                    epochs=1,
                    validation_data = validation_generator,
                    verbose = 1)
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

###############################################################

# #Check model accuracy on the test data
# _, acc = model.evaluate(X_test, y_test)
# print("Accuracy = ", (acc * 100.0), "%")


In [ ]:
test_loss, test_acc = model.evaluate(test_generator, steps= test_generator.samples // batch_size, verbose=1)
print('test acc:', test_acc)

In [ ]:
fuegoen=[]
from matplotlib.patches import Rectangle #To add a rectangle overlay to the image
from skimage.feature.peak import peak_local_max  #To detect hotspots in 2D images. 
def plot_heatmap(filename,orig_img,img):
    fig = plt.figure(figsize=[1,1])
    # This is to get rid of the axes and only get the picture 
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    #Get output from the last conv. layer
    vgg = Model(inputs =model.input, outputs = (model.output,model.get_layer("block5_conv3").output))
  
    pred,last_conv_output = vgg.predict(np.expand_dims(img, axis=0))
    last_conv_output = np.squeeze(last_conv_output)

    pred_class = np.argmax(pred)
    if pred_class == 1:
       plt.imshow(orig_img)
       plt.savefig(filename, dpi=500, bbox_inches='tight',pad_inches=0)
       plt.close()
       return
    fuegoen.append(filename)
    #Get weights for all classes from the prediction layer
    last_layer_weights = model.layers[-1].get_weights()[0] #Prediction layer
    #Get weights for the predicted class.
    last_layer_weights_for_pred = last_layer_weights[:, pred_class]

    #Upsample/resize the last conv. output to same size as original image
    h = int(img.shape[0]/last_conv_output.shape[0])
    w = int(img.shape[1]/last_conv_output.shape[1])
    upsampled_last_conv_output = scipy.ndimage.zoom(last_conv_output, (h, w, 1), order=1)
    
    heat_map = np.dot(upsampled_last_conv_output.reshape((img.shape[0]*img.shape[1], 512)), 
                 last_layer_weights_for_pred).reshape(img.shape[0],img.shape[1])
    
    #Since we have a lot of dark pixels where the edges may be thought of as 
    #high anomaly, let us drop all heat map values in this region to 0.
    # #This is an optional step based on the image. 
    # heat_map[img[:,:,0] == 0] = 0  #All dark pixels outside the object set to 0


    #Detect peaks (hot spots) in the heat map. We will set it to detect maximum 5 peaks.
    #with rel threshold of 0.5 (compared to the max peak). 
    peak_coords = peak_local_max(heat_map, num_peaks=5, threshold_rel=0.5, min_distance=10) 
    plt.imshow(orig_img)
    # plt.imshow(heat_map, cmap='jet', alpha=0.20)
    for i in range(0,peak_coords.shape[0]):
        y = peak_coords[i,0]
        x = peak_coords[i,1]
        plt.gca().add_patch(Rectangle((x-25, y-25), 50,50,linewidth=1,edgecolor='r',facecolor='none'))

    plt.savefig(filename, dpi=500, bbox_inches='tight',pad_inches=0)
    
    # Here we close the image because otherwise we get a warning saying that the image stays
    # open and consumes memory
    plt.close()
    
    return

In [ ]:
img = cv2.imread('/content/drive/MyDrive/Fire Dataset PCD/Train/Fire/77.jpg', 1)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = Image.fromarray(img, 'RGB')
img = img.resize((224, 224))
img = np.array(img)
plt.imshow(img)

In [ ]:
img_tensor = np.expand_dims(img, axis=0)
preprocessed_img = preprocess_input(img_tensor)
preprocessed_img = np.squeeze(preprocessed_img)
heat_map =plot_heatmap('0',img,preprocessed_img)

In [ ]:
import cv2
vidcap = cv2.VideoCapture('/content/drive/MyDrive/test2.mp4')
success, image = vidcap.read()
count = 0
while success:
  cv2.imwrite("/content/fotos/%d.jpg" % count, image)    
  success, image = vidcap.read()
  if count%200 == 0:
    print('Saved image ', count)
  count += 1
print(count)


In [ ]:

video_gen=datagen.flow_from_directory(
    directory='/content/a/',
    batch_size=batch_size,
    shuffle=False,
    target_size= target_size)

In [ ]:
preds = model.predict(video_gen)

In [ ]:
#Genera imagenes con las predicciones y last_convs
fig = plt.figure(figsize=[1,1])
# This is to get rid of the axes and only get the picture 
ax = fig.add_subplot(111)
ax.axes.get_xaxis().set_visible(False)
ax.axes.get_yaxis().set_visible(False)
ax.set_frame_on(False)
#Get output from the last conv. layer
fuegoen=[]
for i in range(len(preds)):
  pred,last_conv_output = preds
  last_conv_output = np.squeeze(last_conv_output)
  pred_class = np.argmax(pred)
  if pred_class == 1:
      plt.imshow(orig_img)
      plt.savefig(filename, dpi=500, bbox_inches='tight',pad_inches=0)
      plt.close()
  else:
    fuegoen.append(filename)
    #Get weights for all classes from the prediction layer
    last_layer_weights = model.layers[-1].get_weights()[0] #Prediction layer
    #Get weights for the predicted class.
    last_layer_weights_for_pred = last_layer_weights[:, pred_class]

    #Upsample/resize the last conv. output to same size as original image
    h = int(img.shape[0]/last_conv_output.shape[0])
    w = int(img.shape[1]/last_conv_output.shape[1])
    upsampled_last_conv_output = scipy.ndimage.zoom(last_conv_output, (h, w, 1), order=1)

    heat_map = np.dot(upsampled_last_conv_output.reshape((img.shape[0]*img.shape[1], 512)), 
                  last_layer_weights_for_pred).reshape(img.shape[0],img.shape[1])

    # #Since we have a lot of dark pixels where the edges may be thought of as 
    # #high anomaly, let us drop all heat map values in this region to 0.
    # #This is an optional step based on the image. 
    # heat_map[img[:,:,0] == 0] = 0  #All dark pixels outside the object set to 0

    #Detect peaks (hot spots) in the heat map. We will set it to detect maximum 5 peaks.
    #with rel threshold of 0.5 (compared to the max peak). 
    peak_coords = peak_local_max(heat_map, num_peaks=5, threshold_rel=0.5, min_distance=10) 
    plt.imshow(orig_img)
    # plt.imshow(heat_map, cmap='jet', alpha=0.20)
    for i in range(0,peak_coords.shape[0]):
        y = peak_coords[i,0]
        x = peak_coords[i,1]
        plt.gca().add_patch(Rectangle((x-25, y-25), 50,50,linewidth=1,edgecolor='r',facecolor='none'))

    plt.savefig(filename, dpi=500, bbox_inches='tight',pad_inches=0)

    # Here we close the image because otherwise we get a warning saying that the image stays
    # open and consumes memory
    plt.close()



In [ ]:
import cv2
vidcap = cv2.VideoCapture('/content/drive/MyDrive/test2.mp4')
success, image = vidcap.read()
count = 0
while success:
  # image = np.resize(image,(224, 224))
  image = Image.fromarray(image, 'RGB')
  image = image.resize((224, 224))
  image = np.array(image)
  # plt.imshow(image)
  img_tensor = np.expand_dims(image, axis=0)
  preprocessed_img = preprocess_input(img_tensor)
  preprocessed_img = np.squeeze(preprocessed_img)
  plot_heatmap('/content/result/'+str(count),image,preprocessed_img)
  # cv2.imwrite("/content/video_images4/image_%d.jpg" % count, image)    
  success, image = vidcap.read()
  if count%20 == 0:
    print('Saved image ', count)
  count += 1



In [ ]:
fuegoen

In [ ]:
import cv2
import os

image_folder = '/content/result'
video_name = 'final.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
len_images = len(images)
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'DIVX'), 30, (width,height))

i = 0
for image in images:
    video.write(cv2.imread(os.path.join(image_folder, str(i)+'.png')))
    i+=1

cv2.destroyAllWindows()
video.release()